In [1]:
import cv2
sys.path

ModuleNotFoundError: No module named 'cv2'

In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset, random_split
from torch import nn
import numpy as np
import sys
import matplotlib.pyplot as plt
import math
import time
import random
import sklearn.metrics as perf
import os
# import cv2
import torch_pruning as tp
import copy, time
import torch.nn.utils.prune as prune

from models.models import MTLClassifier, AgeRegressor, GenderClassifier, EthnicityClassifier
from utils.data import FacesDataset, data_transform
from utils.training import train_mtl_model, train_age_model, train_gender_model, train_ethnicity_model, train_mtl_model_individual
from utils.evaluation import run_evaluation, show_example_predictions
use_gpu = False

In [2]:
### Load in the data
folder = '../UTKFace'
transform = data_transform()
dataset = FacesDataset(folder=folder, transform=transform)

In [3]:
### Set up train and val datasets and loaders
train_len = int(len(dataset)*0.8)
val_len = len(dataset) - train_len
train_dataset, val_dataset = random_split(dataset, [train_len, val_len], torch.Generator().manual_seed(8))

train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=16, shuffle=False)

PRUNING CONV Layers

In [4]:
def prune_model(model, PRUNING_PERCENT=0.2):
    DG = tp.DependencyGraph()
    DG.build_dependency(model, example_inputs=torch.randn(1,3,224,224))
    strategy = tp.strategy.L1Strategy() 
    for name, module in model.named_modules():
        # if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
        if isinstance(module, torch.nn.Conv2d):
            pruning_idxs = strategy(module.weight, amount=PRUNING_PERCENT) # or manually selected pruning_idxs=[2, 6, 9, ...]
            pruning_plan = DG.get_pruning_plan(module, tp.prune_conv, idxs=pruning_idxs )
            pruning_plan.exec()
    return model

In [5]:
def prune_other_tasks(model, task1, task2, PRUNING_PERCENT = 0.1):
    DG = tp.DependencyGraph()
    DG.build_dependency(model, example_inputs=torch.randn(1,3,224,224))
    strategy = tp.strategy.L1Strategy() 
    for name, module in model.named_modules():
        # if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
        if(task1 in name or task2 in name): 
            if isinstance(module, torch.nn.Linear):
                pruning_idxs = strategy(module.weight, amount=1) # or manually selected pruning_idxs=[2, 6, 9, ...]
                pruning_plan = DG.get_pruning_plan(module, tp.prune_linear, idxs=pruning_idxs )
                pruning_plan.exec()
                
            
        if isinstance(module, torch.nn.Conv2d):
            pruning_idxs = strategy(module.weight, amount=PRUNING_PERCENT) # or manually selected pruning_idxs=[2, 6, 9, ...]
            pruning_plan = DG.get_pruning_plan(module, tp.prune_conv, idxs=pruning_idxs )

            pruning_plan.exec()
    return model

In [10]:
#FINE TUNING:
def fine_tune_ethnicity(model):
    model = model.cuda()
    num_epochs = 5
    age_coeff = 0.004
    gender_coeff = 2
    ethni_coeff = 1
    age_criterion = nn.MSELoss()
    gender_criterion = nn.CrossEntropyLoss()
    ethni_criterion = nn.CrossEntropyLoss()

    optimizer = torch.optim.Adam(model.parameters())
    
    train_mtl_model_individual(num_epochs=num_epochs, model=model, optimizer=optimizer,
                    train_loader=train_loader, val_loader=val_loader,
                    age_criterion=age_criterion, gender_criterion=gender_criterion, ethni_criterion=ethni_criterion,
                    age_coeff=age_coeff, gender_coeff=gender_coeff, ethni_coeff=ethni_coeff, save=False, isAge=False, isGender=False)
    
#FINE TUNING:
def fine_tune_gender(model):
    model = model.cuda()
    num_epochs = 3
    age_coeff = 0.004
    gender_coeff = 2
    ethni_coeff = 1
    age_criterion = nn.MSELoss()
    gender_criterion = nn.CrossEntropyLoss()
    ethni_criterion = nn.CrossEntropyLoss()

    optimizer = torch.optim.Adam(model.parameters())
    
    train_mtl_model_individual(num_epochs=num_epochs, model=model, optimizer=optimizer,
                    train_loader=train_loader, val_loader=val_loader,
                    age_criterion=age_criterion, gender_criterion=gender_criterion, ethni_criterion=ethni_criterion,
                    age_coeff=age_coeff, gender_coeff=gender_coeff, ethni_coeff=ethni_coeff, save=False, isAge=False, isEthnicity=False)
#FINE TUNING:
def fine_tune_age(model):
    model = model.cuda()
    num_epochs = 5
    age_coeff = 0.004
    gender_coeff = 2
    ethni_coeff = 1
    age_criterion = nn.MSELoss()
    gender_criterion = nn.CrossEntropyLoss()
    ethni_criterion = nn.CrossEntropyLoss()

    optimizer = torch.optim.Adam(model.parameters())
    
    train_mtl_model_individual(num_epochs=num_epochs, model=model, optimizer=optimizer,
                    train_loader=train_loader, val_loader=val_loader,
                    age_criterion=age_criterion, gender_criterion=gender_criterion, ethni_criterion=ethni_criterion,
                    age_coeff=age_coeff, gender_coeff=gender_coeff, ethni_coeff=ethni_coeff, save=False, isGender=False, isEthnicity=False)

In [7]:
#FINE TUNING:
def fine_tune(model):
    model = model.cuda()
    num_epochs = 5
    age_coeff = 0.004
    gender_coeff = 2
    ethni_coeff = 1
    age_criterion = nn.MSELoss()
    gender_criterion = nn.CrossEntropyLoss()
    ethni_criterion = nn.CrossEntropyLoss()

    optimizer = torch.optim.Adam(model.parameters())
    
    train_mtl_model(num_epochs=num_epochs, model=model, optimizer=optimizer,
                    train_loader=train_loader, val_loader=val_loader,
                    age_criterion=age_criterion, gender_criterion=gender_criterion, ethni_criterion=ethni_criterion,
                    age_coeff=age_coeff, gender_coeff=gender_coeff, ethni_coeff=ethni_coeff, save=False)

In [8]:
def get_inference_time(model):
    if use_gpu:
        model = model.cuda()
    model.eval()
    with torch.no_grad():
        for i, (img, age, gender, ethnicity) in enumerate(val_loader):
              if use_gpu:
                img = img.cuda()
                age = age.float().cuda()
                gender = gender.long().cuda()
                ethnicity = ethnicity.long().cuda()
              
              start = time.time()
              # Get outputs
              age_output, gender_output, ethnicity_output = model(img)
              age_output = age_output.squeeze(1)
              gender_output = gender_output
              ethnicity_output = ethnicity_output

              # Get predictions
              age_pred = age_output
              gender_pred = torch.argmax(gender_output, axis=1)
              ethnicity_pred = torch.argmax(ethnicity_output, axis=1)
              end = time.time()

              inference_latency = end-start
              print("Time to predict:", inference_latency)

              return inference_latency

BASIC PRUNING CODE:

In [4]:
PATH = '../models/mtl_face_model_v1.pt'
ITER_PRUNING = 25
PRUNING_PERCENT = 0.01

model = MTLClassifier()
model.load_state_dict(torch.load('../models/mtl_face_model_v1.pt'))
if use_gpu:
  model = model.cuda()
model.eval()

MTLClassifier(
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 

In [8]:
tasks = ['age', 'gender', 'ethnicity']
mtl_model = True

for idx_prune in range(ITER_PRUNING):
        print(f"\n\nIteration {idx_prune+1} - Pruning {PRUNING_PERCENT*100}% of the least important neurons in every conv")
#         model1 = copy.deepcopy(model)
        model = MTLClassifier()
        model.load_state_dict(torch.load('../models/mtl_face_model_v1.pt'))
        if use_gpu:
          model = model.cuda()
        model.eval()
        if not use_gpu:
            device = torch.device("cpu")
            model.to(device)
        print(f"\nOriginal model inf time: {get_inference_time(model)}")
        pruned_model = prune_model(model, PRUNING_PERCENT)
        if use_gpu:
            pruned_model = pruned_model.cuda()
        fine_tune(pruned_model)
        if not use_gpu:
            device = torch.device("cpu")
            pruned_model.to(device)
        pruned_model.eval()
        inf_time = get_inference_time(pruned_model)
        score_dict = run_evaluation(pruned_model, val_loader, tasks, mtl_model)
        
        f = open("pruned_models/model_data.txt", "a")
        s = f'mtl_face_model_{idx_prune+1}.pth,{score_dict["age"][1]},{score_dict["gender"][1]},{score_dict["ethnicity"][1]},{inf_time}\n'
        f.write(s)
        f.close()
#         torch.save(model.state_dict(),f"pruned_models/mtl_face_model_{idx_prune+16}.pt")
        torch.save(pruned_model, f"pruned_models/mtl_face_model_{idx_prune+1}.pth")
        PRUNING_PERCENT+=0.02




Iteration 1 - Pruning 1.0% of the least important neurons in every conv
Time to predict: 0.06879711151123047

Original model inf time: 0.06879711151123047
Epoch 0, val loss: inf -> 0.09000, train loss: 0.05015
Epoch 0, age val loss: 0.01569, gender val loss: 0.03021, ethnicity val loss: 0.04409

Epoch 1, val loss: 0.11165, train loss: 0.04459
Epoch 1, age val loss: 0.01790, gender val loss: 0.04728, ethnicity val loss: 0.04647

Epoch 2, val loss: 0.09000 -> 0.08416, train loss: 0.04283
Epoch 2, age val loss: 0.01778, gender val loss: 0.03003, ethnicity val loss: 0.03635

Epoch 3, val loss: 0.10017, train loss: 0.03744
Epoch 3, age val loss: 0.01596, gender val loss: 0.04197, ethnicity val loss: 0.04225

Epoch 4, val loss: 0.10151, train loss: 0.03365
Epoch 4, age val loss: 0.01654, gender val loss: 0.04522, ethnicity val loss: 0.03975

Time to predict: 0.07190465927124023


Iteration 2 - Pruning 3.0% of the least important neurons in every conv
Time to predict: 0.06346511840820312

O

Epoch 1, val loss: 0.08965 -> 0.08747, train loss: 0.04974
Epoch 1, age val loss: 0.01716, gender val loss: 0.03134, ethnicity val loss: 0.03897

Epoch 2, val loss: 0.11191, train loss: 0.04667
Epoch 2, age val loss: 0.01650, gender val loss: 0.05120, ethnicity val loss: 0.04421

Epoch 3, val loss: 0.10578, train loss: 0.04016
Epoch 3, age val loss: 0.02174, gender val loss: 0.04492, ethnicity val loss: 0.03913

Epoch 4, val loss: 0.09257, train loss: 0.03499
Epoch 4, age val loss: 0.01735, gender val loss: 0.03279, ethnicity val loss: 0.04244

Time to predict: 0.04285454750061035


Iteration 11 - Pruning 20.999999999999996% of the least important neurons in every conv
Time to predict: 0.05873441696166992

Original model inf time: 0.05873441696166992
Epoch 0, val loss: inf -> 0.09225, train loss: 0.05974
Epoch 0, age val loss: 0.01621, gender val loss: 0.03978, ethnicity val loss: 0.03627

Epoch 1, val loss: 0.09225 -> 0.08913, train loss: 0.05032
Epoch 1, age val loss: 0.01779, gender

Epoch 2, val loss: 0.09126, train loss: 0.05433
Epoch 2, age val loss: 0.01913, gender val loss: 0.03229, ethnicity val loss: 0.03984

Epoch 3, val loss: 0.09802, train loss: 0.04771
Epoch 3, age val loss: 0.01939, gender val loss: 0.03793, ethnicity val loss: 0.04071

Epoch 4, val loss: 0.09079, train loss: 0.04440
Epoch 4, age val loss: 0.01838, gender val loss: 0.03267, ethnicity val loss: 0.03974

Time to predict: 0.026717424392700195


Iteration 20 - Pruning 39.00000000000001% of the least important neurons in every conv
Time to predict: 0.060517311096191406

Original model inf time: 0.060517311096191406
Epoch 0, val loss: inf -> 0.08718, train loss: 0.09107
Epoch 0, age val loss: 0.01970, gender val loss: 0.02743, ethnicity val loss: 0.04005

Epoch 1, val loss: 0.08718 -> 0.08061, train loss: 0.06780
Epoch 1, age val loss: 0.01782, gender val loss: 0.02568, ethnicity val loss: 0.03711

Epoch 2, val loss: 0.08149, train loss: 0.05785
Epoch 2, age val loss: 0.01861, gender val loss

In [12]:
# Pruning for gender task
ITER_PRUNING = 10
PRUNING_PERCENT = 0.05
mtl_model = True
  

tasks = ["gender"]
for idx_prune in range(ITER_PRUNING):
        print(f"\n\nIteration {idx_prune+1} - Pruning {PRUNING_PERCENT*100}% of the least important neurons in every conv")
        model = MTLClassifier()
        model.load_state_dict(torch.load('../models/mtl_face_model_v1.pt'))
        if use_gpu:
          model = model.cuda()
        model.eval()
        if not use_gpu:
            device = torch.device("cpu")
            model.to(device)
        print(f"\nOriginal model inf time: {get_inference_time(model)}")
        pruned_model = prune_other_tasks(model, "age", "ethnicity", PRUNING_PERCENT)

        if use_gpu:
            pruned_model = pruned_model.cuda()
        fine_tune_gender(pruned_model)
        if not use_gpu:
            device = torch.device("cpu")
            pruned_model.to(device)
        pruned_model.eval()
        inf_time = get_inference_time(pruned_model)
        score_dict = run_evaluation(pruned_model, val_loader, tasks, mtl_model)
        print(score_dict)
        f = open("pruned_models/model_data_gender.txt", "a")
        s = f'mtl_face_model_gender_{idx_prune+1}.pth,{score_dict["gender"][1]},{inf_time}\n'
        f.write(s)
        f.close()
#         torch.save(model.state_dict(),f"pruned_models/mtl_face_model_{idx_prune+16}.pt")
        torch.save(pruned_model, f"pruned_models/mtl_face_model_gender_{idx_prune+1}.pth")
        PRUNING_PERCENT+=0.05



Iteration 1 - Pruning 5.0% of the least important neurons in every conv
Time to predict: 0.0635080337524414

Original model inf time: 0.0635080337524414
Epoch 0, val loss: 0.00000, train loss: 0.01705

Epoch 1, val loss: 0.00000, train loss: 0.01566

Epoch 2, val loss: 0.00000, train loss: 0.01202

Time to predict: 0.06020498275756836
{'gender': (0.9228282311199376, 0.9234338747099768)}


Iteration 2 - Pruning 10.0% of the least important neurons in every conv
Time to predict: 0.06328058242797852

Original model inf time: 0.06328058242797852
Epoch 0, val loss: 0.00000, train loss: 0.01859

Epoch 1, val loss: 0.00000, train loss: 0.01557

Epoch 2, val loss: 0.00000, train loss: 0.01340

Time to predict: 0.05147957801818848
{'gender': (0.9272071242131122, 0.9274414680447163)}


Iteration 3 - Pruning 15.000000000000002% of the least important neurons in every conv
Time to predict: 0.05848431587219238

Original model inf time: 0.05848431587219238
Epoch 0, val loss: 0.00000, train loss: 0

In [9]:
# Pruning for age task
ITER_PRUNING = 10
PRUNING_PERCENT = 0.05
mtl_model = True
  

tasks = ["age"]
for idx_prune in range(ITER_PRUNING):
        print(f"\n\nIteration {idx_prune+1} - Pruning {PRUNING_PERCENT*100}% of the least important neurons in every conv")
        model = MTLClassifier()
        model.load_state_dict(torch.load('../models/mtl_face_model_v1.pt'))
        if use_gpu:
          model = model.cuda()
        model.eval()
        if not use_gpu:
            device = torch.device("cpu")
            model.to(device)
        print(f"\nOriginal model inf time: {get_inference_time(model)}")
        pruned_model = prune_other_tasks(model, "gender", "ethnicity", PRUNING_PERCENT)

        if use_gpu:
            pruned_model = pruned_model.cuda()
        fine_tune_age(pruned_model)
        if not use_gpu:
            device = torch.device("cpu")
            pruned_model.to(device)
        pruned_model.eval()
        inf_time = get_inference_time(pruned_model)
        score_dict = run_evaluation(pruned_model, val_loader, tasks, mtl_model)
        print(score_dict)
        f = open("pruned_models/model_data_age.txt", "a")
        s = f'mtl_face_model_age_{idx_prune+1}.pth,{score_dict["age"][1]},{inf_time}\n'
        f.write(s)
        f.close()
#         torch.save(model.state_dict(),f"pruned_models/mtl_face_model_{idx_prune+16}.pt")
        torch.save(pruned_model, f"pruned_models/mtl_face_model_age_{idx_prune+1}.pth")
        PRUNING_PERCENT+=0.05



Iteration 1 - Pruning 5.0% of the least important neurons in every conv
Time to predict: 0.0689079761505127

Original model inf time: 0.0689079761505127
Epoch 0, val loss: 0.00000, train loss: 0.01543

Epoch 1, val loss: 0.00000, train loss: 0.01257

Epoch 2, val loss: 0.00000, train loss: 0.01005

Epoch 3, val loss: 0.00000, train loss: 0.00936

Epoch 4, val loss: 0.00000, train loss: 0.00813

Time to predict: 0.06615352630615234
{'age': (7.728669, 0.848694845224829)}


Iteration 2 - Pruning 10.0% of the least important neurons in every conv
Time to predict: 0.062120914459228516

Original model inf time: 0.062120914459228516
Epoch 0, val loss: 0.00000, train loss: 0.01598

Epoch 1, val loss: 0.00000, train loss: 0.01234

Epoch 2, val loss: 0.00000, train loss: 0.01033

Epoch 3, val loss: 0.00000, train loss: 0.00950

Epoch 4, val loss: 0.00000, train loss: 0.00893

Time to predict: 0.05163979530334473
{'age': (7.980887, 0.8386583066058323)}


Iteration 3 - Pruning 15.000000000000002

In [ ]:
# Pruning for ethnicity task
ITER_PRUNING = 10
PRUNING_PERCENT = 0.05
mtl_model = True
  

tasks = ["ethnicity"]
for idx_prune in range(ITER_PRUNING):
        print(f"\n\nIteration {idx_prune+1} - Pruning {PRUNING_PERCENT*100}% of the least important neurons in every conv")
        model = MTLClassifier()
        model.load_state_dict(torch.load('../models/mtl_face_model_v1.pt'))
        if use_gpu:
          model = model.cuda()
        model.eval()
        if not use_gpu:
            device = torch.device("cpu")
            model.to(device)
        print(f"\nOriginal model inf time: {get_inference_time(model)}")
        pruned_model = prune_other_tasks(model, "age", "gender", PRUNING_PERCENT)

        if use_gpu:
            pruned_model = pruned_model.cuda()
        fine_tune_ethnicity(pruned_model)
        if not use_gpu:
            device = torch.device("cpu")
            pruned_model.to(device)
        pruned_model.eval()
        inf_time = get_inference_time(pruned_model)
        score_dict = run_evaluation(pruned_model, val_loader, tasks, mtl_model)
        print(score_dict)
        f = open("pruned_models/model_data_ethnicity.txt", "a")
        s = f'mtl_face_model_age_{idx_prune+1}.pth,{score_dict["ethnicity"][1]},{inf_time}\n'
        f.write(s)
        f.close()
#         torch.save(model.state_dict(),f"pruned_models/mtl_face_model_{idx_prune+16}.pt")
        torch.save(pruned_model, f"pruned_models/mtl_face_model_ethnicity_{idx_prune+1}.pth")
        PRUNING_PERCENT+=0.05



Iteration 1 - Pruning 5.0% of the least important neurons in every conv
Time to predict: 0.0605769157409668

Original model inf time: 0.0605769157409668
Epoch 0, val loss: 0.00000, train loss: 0.02597

Epoch 1, val loss: 0.00000, train loss: 0.02170

Epoch 2, val loss: 0.00000, train loss: 0.01874

Epoch 3, val loss: 0.00000, train loss: 0.01725

Epoch 4, val loss: 0.00000, train loss: 0.01432

Time to predict: 0.05576324462890625
{'ethnicity': (0.7182205739854156, 0.8192364480067497)}


Iteration 2 - Pruning 10.0% of the least important neurons in every conv
Time to predict: 0.062483787536621094

Original model inf time: 0.062483787536621094
Epoch 0, val loss: 0.00000, train loss: 0.02675

Epoch 1, val loss: 0.00000, train loss: 0.02225

Epoch 2, val loss: 0.00000, train loss: 0.01907

Epoch 3, val loss: 0.00000, train loss: 0.01626

Epoch 4, val loss: 0.00000, train loss: 0.01481

Time to predict: 0.04430079460144043
{'ethnicity': (0.722703667920865, 0.8072136680025311)}


Iteratio